# 단순한 계층 구현하기

### 곱셈 계층

In [1]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x*y
        return out
    
    def backward(self, dout):
        dx = dout*self.y
        dy = dout*self.x
        return dx, dy

#### 사과문제 예시

In [2]:
apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

#순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

price

220.00000000000003

In [3]:
#역전파
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)
print(dapple, dapple_num, dtax)

2.2 110.00000000000001 200


### 덧셈 계층

In [4]:
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        return x + y
    
    def backward(self, dout):
        return dout*1, dout*1

#### 사과문제 예시

In [5]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

#계층들
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

#순전파
apple_price = mul_apple_layer.forward(apple, apple_num)
orange_price = mul_orange_layer.forward(orange, orange_num)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)
price = mul_tax_layer.forward(all_price, tax)
print(price)

#역전파
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)
print(dapple_num, dapple, dorange, dorange_num, dtax)

715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


# 활성화 함수 계층 구현하기

### ReLU 계층

In [6]:
class ReLU:
    def __init__(slef):
        self.mask = None
        
    def forward(self,x):
        self.mask = (x <=0)
        out = x.copy()
        out[self.mask] = 0
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx

In [7]:
import numpy as np

### Sigmoid 계층 

In [8]:
class Sigmoid:
    def __init__(self):
        self.out = None
        
    def forward(self,x):
        out = 1/ (1+np.exp(-x))
        self.out = out
        return out
    
    def backward(self,dout):
        dx = dout*(1.0-slef.out)*self.out
        return dx

### Affine 계층

In [10]:
#행렬의 내적, affine transform
X = np.random.rand(2)  #입력
W = np.random.rand(2,3)  #가중치
B = np.random.rand(3)  #편향

print(X.shape)
print(W.shape)
print(B.shape)

Y = np.dot(X,W) + B

(2,)
(2, 3)
(3,)


### 배치용 Affine 계층

In [11]:
# X의 형상이 (2,) 에서 (N,2)로 바뀜
# N=2인 경우 bias 더하는 것 예시

X_dot_W = np.array([[0, 0, 0],[10,10,10]])
B = np.array([1,2,3])

X_dot_W + B

array([[ 1,  2,  3],
       [11, 12, 13]])

In [12]:
# 역전파 편향 예시, N = 2 인 경우

dY = np.array([[1,2,3],[4,5,6]])
print(dY)

dB = np.sum(dY, axis=0)
dB

[[1 2 3]
 [4 5 6]]


array([5, 7, 9])

In [13]:
#Affine 구현

class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self,x):
        self.x = x
        out = np.dot(x, self.W) +self.b
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        
        return dx

## softmax-with-loss 계층

In [14]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None #손실
        self.y = None #softmax의 출력
        self.t = None #정답 레이블(one-hot vector)
        
    def forward(self,x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout = 1):
        batch_size = self.t.shape[0]
        dx = (self.y-self.t) / batch_size
        
        return dx

## 오차전역법 구현하기

In [28]:
#2층 신경망 class

import sys, os
sys.path.append(os.pardir)
import numpy as np

from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:
    
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        #가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        #계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        
        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
            
        return x
    
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy (self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis = 1)
        if t.ndim != 1:
            t = np.argmax(t, axis = 1)
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x,t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads
    
    def gradient(self, x,  t):
        #순전파
        self.loss(x,t)
        #역전파
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        #결과 저장
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        
        return grads

#### 오차역전파법으로 구한 기울기 검증하기

In [26]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

#데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize = True, one_hot_label = True)

network = TwoLayerNet(784, 50, 10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key, ': ', diff)
print("수치 미분과 오차역전파법의 기울기 차이가 매우 작음")

W1 :  4.2715474561506737e-10
b1 :  2.3487609834393474e-09
W2 :  5.416528056380663e-09
b2 :  1.3976275235888514e-07
수치 미분과 오차역전파법의 기울기 차이가 매우 작음


### 오차역전파법을 사용한 학습 구현하기

In [29]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

#데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize = True, one_hot_label = True)

network = TwoLayerNet(784, 50, 10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size/ batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    #오차역전파법
    grad = network.gradient(x_batch, t_batch)
    
    #갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate*grad[key]
        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.11286666666666667 0.1097
0.90445 0.9059
0.9238333333333333 0.9261
0.9357 0.9347
0.9444333333333333 0.9427
0.9519666666666666 0.9495
0.9556 0.9519
0.9613166666666667 0.9575
0.9653666666666667 0.9604
0.9689 0.9629
0.9702666666666667 0.9625
0.9715666666666667 0.964
0.9753 0.9667
0.9750166666666666 0.9674
0.9772833333333333 0.9677
0.9788666666666667 0.969
0.9793833333333334 0.9686
